In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('Weather Dataset\Wheater_Classification\weatherHistory (1).csv') 
df2 = pd.read_csv('Weather Dataset\Time-series-cuaca.csv')

In [3]:
df.describe()

,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars)
count,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.0,96453.000000
mean,11.932678,10.855029,0.734899,10.810640,187.509232,10.347325,0.0,1003.235956
std,9.551546,10.696847,0.195473,6.913571,107.383428,4.192123,0.0,116.969906
min,-21.822222,-27.716667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,4.688889,2.311111,0.600000,5.828200,116.000000,8.339800,0.0,1011.900000
50%,12.000000,12.000000,0.780000,9.965900,180.000000,10.046400,0.0,1016.450000
75%,18.838889,18.838889,0.890000,14.135800,290.000000,14.812000,0.0,1021.090000
max,39.905556,39.344444,1.000000,63.852600,359.000000,16.100000,0.0,1046.380000


In [4]:
df = df.drop(['Apparent Temperature (C)','Visibility (km)', 'Loud Cover', 'Pressure (millibars)','Precip Type','Daily Summary'], axis=1)

In [5]:
def transform_date(df):
    df["Formatted Date"] = pd.to_datetime(df["Formatted Date"], utc=True).dt.date
    return df


In [6]:
df = transform_date(df)

In [7]:
df = df.drop(['Formatted Date'], axis=1)

In [8]:
def transform_df(df1,df2):
    old_min = df1.min()
    old_max = df1.max()
    old_mean = df1.mean()
    old_std = df1.std()
    
    new_min = df2.min()
    new_max = df2.max()
    new_mean = df2.mean()
    new_std = df2.std()
    
    transformed_df = ((df1 - old_min) * (new_max - new_min) / (old_max - old_min)) + new_min
    # transformed_df = ((transformed_df - old_mean) * (new_std / old_std)) + new_mean
    
    return transformed_df

In [9]:
df["Temperature (C)"] = transform_df(df["Temperature (C)"],df2["Tavg"])
df["Humidity"] = transform_df(df["Humidity"],df2["RH_avg"])
df["Wind Speed (km/h)"] = transform_df(df["Wind Speed (km/h)"],df2["ff_x"])
df["Wind Bearing (degrees)"] = transform_df(df["Wind Bearing (degrees)"],df2["ddd_x"])

In [10]:
df.describe()

,Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees)
count,96453.000000,96453.000000,96453.000000,96453.000000
mean,25.787083,80.624140,4.200980,192.808444
std,1.609261,11.337419,1.407561,104.691365
min,20.100000,38.000000,2.000000,10.000000
25%,24.566637,72.800000,3.186586,123.091922
50%,25.798425,83.240000,4.028996,185.487465
75%,26.950653,89.620000,4.877963,292.729805
max,30.500000,96.000000,15.000000,360.000000


In [11]:
label_encoder = LabelEncoder()
df['Summary'] = label_encoder.fit_transform(df['Summary'])

In [12]:
target_counts = df['Summary'].value_counts()
print(target_counts)


19    31733
17    28094
18    16597
6     10890
12     7148
4       528
3       516
5       386
11       86
26       67
16       63
0        54
25       45
13       40
8        39
2        35
24       35
9        34
15       17
10       14
20       10
21        8
14        7
23        4
22        1
7         1
1         1
Name: Summary, dtype: int64


In [13]:
# Menghitung jumlah baris per target label
target_counts = df['Summary'].value_counts()

# Mengurangi target label dengan jumlah baris lebih dari 1200
target_to_reduce = target_counts[(target_counts > 1200) | (target_counts < 20)].index
for target in target_to_reduce:
    target_count = target_counts[target]
    if target_count > 1000:
        scale = 0.97
    else:
        scale = 1.0
    target_df = df[df['Summary'] == target]
    if target_count < 20:
        df = df.drop(target_df.index)
    else:
        target_df = target_df.sample(frac=scale, random_state=42)
        df = df.drop(target_df.index)

# Memperbarui target_counts setelah mengatur ulang jumlah baris
target_counts = df['Summary'].value_counts()

In [14]:
df

,Summary,Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees)
50,18,25.231203,88.46,6.487393,41.197772
80,3,24.811871,91.94,8.283661,350.250696
81,3,24.978481,84.40,8.549168,350.250696
466,17,26.780299,80.34,3.520928,171.838440
501,19,26.301062,79.76,4.838628,135.766017
...,...,...,...,...,...
96297,19,27.626451,65.84,3.953606,188.412256
96343,4,26.744730,90.20,8.306606,331.727019
96357,19,26.190613,91.94,2.698185,283.955432
96364,17,25.945378,96.00,2.921079,340.501393


In [15]:
target_counts = df['Summary'].value_counts()
print(target_counts)

19    952
17    843
4     528
3     516
18    498
5     386
6     327
12    214
11     86
26     67
16     63
0      54
25     45
13     40
8      39
2      35
24     35
9      34
Name: Summary, dtype: int64


In [16]:
df.describe()

,Summary,Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees)
count,4762.000000,4762.000000,4762.000000,4762.000000,4762.000000
mean,12.257875,25.753057,78.499273,5.819796,205.735184
std,6.961431,1.519076,11.715725,2.566796,104.810080
min,0.000000,20.407011,46.700000,2.000000,10.000000
25%,5.000000,24.646665,70.480000,3.615986,145.515320
50%,17.000000,25.636495,80.340000,4.884518,204.986072
75%,18.000000,26.771875,87.880000,8.391011,304.428969
max,26.000000,30.305310,96.000000,13.387292,360.000000


In [17]:
# Melakukan decode pada kolom target setelah penyesuaian
df['Summary'] = label_encoder.inverse_transform(df['Summary'])

# Mengurutkan ulang label secara alfabetis
sorted_labels = sorted(df['Summary'].unique())

# Membuat dictionary untuk menyimpan keterangan encode baru
encode_labels_new = {}

# Melakukan encoding ulang dengan label yang berurutan
for i, label in enumerate(sorted_labels):
    encode_labels_new[label] = i
    df.loc[df['Summary'] == label, 'Summary'] = i

# # Menyimpan data setelah encode ulang
# df.to_csv('data_encode_ulang.csv', index=False)

# Menampilkan keterangan encode dan jumlah baris per target setelah encode ulang
for target_label, target_encode in encode_labels_new.items():
    count_after = df[df['Summary'] == target_encode].shape[0]
    print("Target Label:", target_label)
    print("Encode Label:", target_encode)
    print("Jumlah Baris:", count_after)
    print()


Target Label: Breezy
Encode Label: 0
Jumlah Baris: 54

Target Label: Breezy and Foggy
Encode Label: 1
Jumlah Baris: 35

Target Label: Breezy and Mostly Cloudy
Encode Label: 2
Jumlah Baris: 516

Target Label: Breezy and Overcast
Encode Label: 3
Jumlah Baris: 528

Target Label: Breezy and Partly Cloudy
Encode Label: 4
Jumlah Baris: 386

Target Label: Clear
Encode Label: 5
Jumlah Baris: 327

Target Label: Drizzle
Encode Label: 6
Jumlah Baris: 39

Target Label: Dry
Encode Label: 7
Jumlah Baris: 34

Target Label: Dry and Partly Cloudy
Encode Label: 8
Jumlah Baris: 86

Target Label: Foggy
Encode Label: 9
Jumlah Baris: 214

Target Label: Humid and Mostly Cloudy
Encode Label: 10
Jumlah Baris: 40

Target Label: Light Rain
Encode Label: 11
Jumlah Baris: 63

Target Label: Mostly Cloudy
Encode Label: 12
Jumlah Baris: 843

Target Label: Overcast
Encode Label: 13
Jumlah Baris: 498

Target Label: Partly Cloudy
Encode Label: 14
Jumlah Baris: 952

Target Label: Windy and Mostly Cloudy
Encode Label: 15


In [22]:
df

,Summary,Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees)
50,13,25.231203,88.46,6.487393,41.197772
80,2,24.811871,91.94,8.283661,350.250696
81,2,24.978481,84.40,8.549168,350.250696
466,12,26.780299,80.34,3.520928,171.838440
501,14,26.301062,79.76,4.838628,135.766017
...,...,...,...,...,...
96297,14,27.626451,65.84,3.953606,188.412256
96343,3,26.744730,90.20,8.306606,331.727019
96357,14,26.190613,91.94,2.698185,283.955432
96364,12,25.945378,96.00,2.921079,340.501393


In [19]:
# df.to_csv("Wheater-Encode-22.csv", index=False)

In [20]:
# # def transform_value(value, old_min, old_max, new_min, new_max):
# #     old_range = old_max - old_min
# #     new_range = new_max - new_min
# #     new_value = ((value - old_min) * new_range / old_range) + new_min
# #     return new_value
# def transform_value(value, old_min, old_max, new_min, new_max, mean):
#     old_range = old_max - old_min
#     new_range = new_max - new_min
    
#     if value == mean:
#         new_value = new_min + (new_max - new_min) / 2
#     else:
#         new_value = ((value - old_min) * new_range / old_range) + new_min
    
#     return new_value
    
#     return new_value

# def transform_Temp(df):
#     OLD_MIN = df["Temperature (C)"].min()
#     OLD_MAX = df["Temperature (C)"].max()
#     NEW_MIN = 14.6
#     NEW_MAX = 37.5
#     MEAN = 25.619143	
#     df["Temperature (C)"] = transform_value(df["Temperature (C)"],OLD_MIN,OLD_MAX,NEW_MIN,NEW_MAX,MEAN)
#     return df

# # def transform_value_RH(value, old_min, old_max, new_min, new_max, mean):
# #     old_range = old_max - old_min
# #     new_range = new_max - new_min
    
# #     if value == mean:
# #         new_value = new_min + (new_max - new_min) / 2
# #     else:
# #         new_value = ((value - old_min) * new_range / old_range) + new_min
    
# #     return new_value

# # def transform_RH(df):
# #     OLD_MIN = df["Humidity"].min()
# #     OLD_MAX = df["Humidity"].max()
# #     NEW_MIN = 0.38
# #     NEW_MAX = 0.96
# #     df["Humidity"] = transform_value_RH(df["Humidity"],OLD_MIN,OLD_MAX,NEW_MIN,NEW_MAX)
# #     return df


In [21]:
# def transform_value(value, old_min, old_max, new_min, new_max):
#     old_range = old_max - old_min
#     new_range = new_max - new_min
#     new_value = ((value - old_min) * new_range / old_range) + new_min
#     return 

# def transform_ApTemp(df):
#     OLD_MIN = df["Apparent Temperature (C)"].min()
#     OLD_MAX = df["Apparent Temperature (C)"].max()
#     NEW_MIN = 10
#     NEW_MAX = 38.5
#     df["Apparent Temperature (C)"] = transform_value(df["Apparent Temperature (C)"],OLD_MIN,OLD_MAX,NEW_MIN,NEW_MAX)
#     return df
